In [1]:
# import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
import json
import sys
import os
import re
import pprint
import time
import copy
from bs4 import BeautifulSoup

In [2]:
# create the data directory, remove the files if they exits
folder_name = "./data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
    for i in os.scandir(folder_name):
        try:
            os.remove(i)
        except:
            continue

In [3]:
# get the data from url
image_prediction_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv"
twitter_archive_enhanced_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/twitter-archive-enhanced.csv"
tweet_json_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/tweet_json.txt"

In [4]:
# get the data by using a function
def get_data(url, path):
    """
    get the data from the url. Return the path of the data
    file
    
    Args:
    (str) url - the data address on the web
    (str) path - the parent path which stores the data
    
    Returns:
    (str) result - the file path which store the data
    """
    
    result = os.path.join(path, os.path.basename(url))
    
    url_request = requests.get(url)
    
    with open(result, "wb") as file:
        file.write(url_request.content)
    # adjust the sleep time to increase requesting interval
    time.sleep(3)
    return result

In [ ]:
# get the datas by using the url and the function
if True:
    image_prediction = get_data(image_prediction_url, folder_name)
    twitter_archive_enhanced = get_data(twitter_archive_enhanced_url, folder_name)
    tweet_json = get_data(tweet_json_url, folder_name)
else:
    image_prediction = "./data/image-predictions.tsv"
    twitter_archive_enhanced = "./data/twitter-archive-enhanced.csv"
    tweet_json = "./data/tweet_json.txt"

In [ ]:
# compare the files, which does just check the files
import filecmp
if False:
    print(filecmp.cmp(image_prediction, "./Test/image-predictions.tsv"), 
          filecmp.cmp(twitter_archive_enhanced, "./Test/twitter-archive-enhanced.csv"),
          filecmp.cmp(tweet_json, "./Test/tweet_json.txt"))

### 1. 收集数据
1. 本次项目中数据已经由 `project` 给出了相关的下载链接，因此先直接使用了 `requests` 库获取相关的数据，保留了原始的文件名。另外因为是对数据获取是使用 `requests` 进行下载，通过完全将数据通过 `binary` 写入文件。为了验证得到数据的准确性，使用了 `filecmp` 对直接下载的数据和 `requests` 得到的数据进行比较。

2. 本次项目中得到了三种类型文件： `csv`, `tsv` 以及 `txt`。前两种文件类型比较容易处理，通过 `pandas` 的相关读取方法即可得到数据内容；但是 `txt` 的文件比较有难度，体现在——a）文件内容其实是 `json` 类型的；b）通过 `open` 的方式读取得到的数据是一个字符串，需要对数据内容进行相关处理。c）对数据进行相关的提取，一方面是因为在实际情况中并非是所有的数据都是必须的。对某些数据需要合适转换才能完成数据完整读取

In [ ]:
# load the datas
twitter_archive_data = pd.read_csv(twitter_archive_enhanced)

image_prediction_data = pd.read_csv(image_prediction, sep="\t")

In [ ]:
tweet_data = pd.DataFrame()
index_count = -1

with open(tweet_json, "rb") as file:
    for i in file.readlines():
        temp = []
        dict_data = json.loads(i)
        temp.append({"id": dict_data["id"], 
                     "created_at": dict_data["created_at"], 
                     "full_text": dict_data["full_text"], 
                     "retweet_count":dict_data["retweet_count"], 
                     "favorite_count":dict_data["favorite_count"],
                     "in_reply_to_user_id": dict_data["in_reply_to_user_id"],
                     "in_reply_to_status_id": dict_data["in_reply_to_status_id"],
                     "source": dict_data["source"]})
        # maybe dict data can't contain the key
        try:
            temp[0]["media_url"] =  dict_data["entities"]["media"][0]["media_url_https"]
        except KeyError:
            temp[0]["media_url"] =  None
            
        try:
            temp[0]["expanded_url"] =  dict_data["extended_entities"]["media"][0]["expanded_url"]
        except KeyError:
            temp[0]["expanded_url"] =  None
            
            
        temp = pd.DataFrame(temp, columns=["id", "created_at", "full_text", "media_url",
                                           "favorite_count", "retweet_count", "in_reply_to_status_id",
                                           "in_reply_to_user_id", "expanded_url", "source"],
                            index=pd.Index([index_count+1]))
        tweet_data = pd.concat([tweet_data, temp])
        
        # increase the index value
        index_count += 1

In [ ]:
# print the keys
def print_fields(column_data, name):
    """
    display the information about the data fields
    
    Args:
    (Array) column_data - the fields about the data
    (str) name - the name of the data variable
    
    Returns:
    (None)
    """    
    print("There are %d fields in the %s:\n" % (len(column_data), name))

    for i, key in enumerate(column_data):
        print("{0:<2}\t{1:<30}".format(i+1, key))

In [ ]:
# image_prediction_data
print_fields(image_prediction_data.columns, "image_prediction_data")

In [ ]:
# twitter_archive_data
print_fields(twitter_archive_data.columns, "twitter_archive_data")

In [ ]:
# tweet_data
print_fields(tweet_data.columns, "tweet_data")

### 2. 评估数据
#### 2.1 对数据的 `field` 进行评估
对 `tsv` 和 `csv` 文件采取了直接使用 `pandas` 读取数据，直接得到了两者的 `field`：其中 `image-prediction.tsv` 中有 `12` 个 `fields`，得到的结果如下：

1. tweet_id : 推特链接中位于 "status/" 后面的一部分
2. jpg_url:预测的图像资源链接
3. img_num : 最可信的预测结果对应的图像编号
4. p1 : 是算法对推特中图片的一号预测
5. p1_conf : 是算法预测的可信度
6. p1_dog : 预测该图片是否属于“狗”
7. p2 : 算法对推特中图片预测的第二种可能性
8. p2_conf : 第二种算法的预测的可信度
9. p2_dog : 第二种算法预测该图片是否属于“狗”
10. p3 : 算法对推特中图片预测的第三种可能性
11. p3_conf : 第三种算法的预测的可信度
12. p3_dog : 第三种算法预测该图片是否属于“狗”


而 `twitter-archive-enhanced.csv` 中有 `17` 个 `fields`，得到的结果如下：

1. tweet_id : 推特链接中位于 "status/" 后面的最后一部分
2. in_reply_to_status_id : 代表是否有回复，如果有保留了 `tweet_id` 的数据，以数字形式保存
3. in_reply_to_user_id : 代表是否有回复，如果有保留了 `tweet_id` 的数据，以字符串形式保存
4. timestamp : 创建时间
5. source : 发送本条 `tweet` 时的设备信息，以 `HTML` 标签样式保存数据
6. text : 发送的 `tweet` 信息，以 `utf-8` 的字符串形式保存数据
7. retweeted_status_id : 第一个转 `tweet` 用户的信息
8. retweeted_status_user_id : 第一个转 `tweet` 用户的信息
9. retweeted_status_timestamp : 第一个转 `tweet` 用户的信息中的时间
10. expanded_urls : `tweet` 的 `entities` 中 `url`
11.	rating_numerator : 评分分数的分子
12.	rating_denominator : 评分分数的分母
13.	name : 狗的品种名称
14.	doggo : 狗是否属于该类型
15.	floofer : 狗是否属于该类型
16.	pupper : 狗是否属于该类型
17.	puppo : 狗是否属于该类型


因为 `tweet_json.txt` 文件需要将数据转化为 `json` 类型，额外提取相关数据信息进行组合。本次提取数据是参考以上两个文件中的已有的 `field` 进行分析，一方面是想通过该数据验证以上数据是否正确，另一方面也是为了验证数据清洗的结果是否合理及正确。具体提取的 `field` 如下：

1. id : 推特链接中位于 "status/" 后面的最后一部分，以数字形式保存
2. created_at : `tweet` 发送时间
3. full_text : 发送的 `tweet` 信息，以 `utf-8` 的字符串形式保存数据
4. media_url : 狗的图像资源链接
5. favorite_count : 表示被 `twitter` 用户喜欢的数量
6. retweet_count : 表示被转发 `tweet` 的数量
7. in_reply_to_status_id : 代表是否有回复，如果有保留了 `tweet_id` 的数据，以数字形式保存
8. in_reply_to_user_id : 代表是否有回复，如果有保留了 `tweet_id` 的数据，以字符串形式保存
9. expanded_url : 
10. source : 发送本条 `tweet` 时的设备信息，以 `HTML` 标签样式保存数据


In [ ]:
pprint.pprint(dict_data)

In [ ]:
def detect_data(df, name):
    """
    detect the data information about missing value, data types
    and unique values
    
    Args:
    (dataframe) df - dataframe storing the data
    (str) name - variable name referring  the data
    
    Returns:
    (None)
    """
    print("The summary information:\n")
    print("There are %d fiels and %d data points in the %s.\n" % (df.shape[1], df.shape[0], name))
    print("=" * 80)
    
    print("The missing_value information about the %s:\n" % name)
    print(df.isnull().sum())
    print("=" * 80)

    print("The unique value about every field:\n")
    
    for i in df.columns:
        print("The %s field with %s dtype has unique values:\n" % (i, df[i].dtypes))
        if len(df[i].unique()) > 5:
            print(",\n".join([str(df[i].unique()[element]) for element in range(5)]))
        else:
            print(",\n".join([str(element) for element in df[i].unique()]))
        print("-" * 80 + "\n")

    print("=" * 80)

In [ ]:
detect_data(image_prediction_data, "image_prediction_data")

In [ ]:
# plot the missing value hist plot
x_axis = image_prediction_data.columns.get_values()
height = image_prediction_data.count() - image_prediction_data.isnull().sum()

plt.figure(figsize=(15, 12))
plt.bar(np.arange(len(x_axis)), height, tick_label=x_axis, facecolor="grey", width=.8)
loc, labels = plt.xticks(rotation=330, fontsize=13)
plt.xlabel("Field", fontsize=16)

plt.hlines(y=2075, xmin=-0.5, xmax=loc.max()+1,colors="r", linestyles="--", linewidth=4,label="Total Data Point:2075")
plt.legend()
plt.title("Fig Unmissing Value Count About Image Prediction", loc="left", fontsize=20)
plt.show()

In [ ]:
detect_data(twitter_archive_data, "twitter_archive_data")

In [ ]:
# plot the missing value hist plot
x_axis = twitter_archive_data.columns.get_values()
height = len(twitter_archive_data) - twitter_archive_data.isnull().sum()
hist_data = pd.DataFrame({"field": height.index.values, "count":height.values}, columns=["field", "count"])

hist_data.plot(x="field", kind="bar", figsize=(20, 17), rot=30, color="grey") #, yticks=np.arange(height.min()-500, height.max()+200, 300))
plt.xlabel("Field", fontsize=16)
loc, labels = plt.xticks( fontsize=10)
plt.ylabel("Total Unmissing Value Count", fontsize=15)
plt.yticks(np.arange(0, height.max()+200, 100), fontsize=14)
plt.title("Fig Unmissing Value Count About Twitter Archive Enhanced", loc="left", fontsize=20)


for x, y, s in zip(loc, [i + 10 for i in height], [str(i) for i in height]):
    plt.text(x-0.2, y, s, fontsize=12)
    
plt.hlines(y=2356, xmin=-0.5, xmax=loc.max()+1,colors="r",
           linestyles="--", linewidth=4,label="Total Data Point:2356")

plt.legend(loc=(0.35, 0.8), fontsize="large")
plt.show()

In [ ]:
detect_data(tweet_data, "tweet_data")

In [ ]:
# plot the missing value hist plot
x_axis = tweet_data.columns.get_values()
height = len(tweet_data) - tweet_data.isnull().sum()
hist_data = pd.DataFrame({"field": height.index.values, "count":height.values}, columns=["field", "count"])

hist_data.plot(x="field", kind="bar", figsize=(20, 13), rot=30, color="grey", legend="") #, yticks=np.arange(height.min()-500, height.max()+200, 300))
plt.xlabel("Field", fontsize=16)
loc, label = plt.xticks( fontsize=10)
plt.ylabel("Total Unmissing Value Count", fontsize=15)
plt.yticks()
plt.xticks(fontsize=15)

for x, y, s in zip(loc, [i + 10 for i in height], [str(i) for i in height]):
    plt.text(x-0.15, y, s, fontsize=15)

plt.title("Fig Unmissing Value Count About Tweet Data", loc="left", fontsize=20)
plt.hlines(y=2352, xmin=-0.5, xmax=loc.max()+1,colors="r", linestyles="--", linewidth=4,label="Total Data Point:2352")
plt.legend(loc=(0.59,0.8), fontsize="xx-large")
plt.show()

### 2. 评估数据
#### 2.2 对数据内容及数据数据类型评估
1. 对 `image_prediction_data` 进行数据内容和数据类型探索，该数据集有 `2075` 个数据且有 	`12` 个 `field`；另外发存在数据类型不合适的情况，该数据集没有缺失值的情况。

2. 对 `twitter_archive_data` 进行数据内容和数据类型探索，该数据集中有 `2356` 个数据且有 `17` 个 `field`；在数据类型不合适的情况以及该数据集某些 `field`  存在缺失值的情况—— `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` 数据缺失严重；另外在 `expanded_urls` 中存在部分数据缺失

3. 对 `tweet_data` 进行数据内容和数据类型探索，该数据集有 `10` 个 `field` 以及 `2352` 个数据点。该数据的主要问题，还是是数据类型不符合 `id`, `created_at`, `in_reply_to_status_id`, `in_reply_to_user_id`, `source`；一个行列单元格包含了多个信息， `full_text` 包括了 `tweet` 信息、评分分数以及链接。

#### 2.3 总体评估结论：
**质量**

1. `image_prediction_data` 数据集

	* `tweet_id` 是 `int64` 的整数类型，实际应当保存为 `object` 类型
	* `p1` 数据中首字母存在大写和小写混用的情况
	* `p2` 数据中首字母存在大写和小写混用的情况
	* `p3` 数据中首字母存在大写和小写混用的情况
	* `p1_conf` 是比例数据，可以确认需要保留的小数点位数
	* `p2_conf` 是比例数据，可以确认需要保留的小数点位数
	* `p3_conf` 是比例数据，可以确认需要保留的小数点位数

2. `twitter_archive_data`	数据集
	
	* `tweet_id` 是 `int64` 的整数类型，实际应当保存为 `object` 类型
	* `in_reply_to_status_id` 是 `float64` 的整数类型，实际应当保存为 `object` 类型
	* `in_reply_to_user_id` 是 `float64` 的整数类型，实际应当保存为 `object` 类型
	* `timestamp` 是 `object` 的对象数据(字符串模式)，实际应当保存为 `datetime` 类型
	* `retweeted_status_id` 是 `float64` 的数值类型数据，实际应当是 `object` 类型
	* `retweeted_status_user_id` 是 `float64` 的数值类型数据，实际应当是 `object` 类型
	* `retweeted_status_timestamp` 是 `object` 的对象数据(字符串模式)，实际应当保存为 `datetime` 类型
	* 存在缺失值: 缺少相关信息，不合适进行处理

3. `tweet_data`	的数据集

	* `id` 是 `int64` 的整数类型，实际应当保存为 `object` 类型
	* `created_at` 是 `object` 的对象数据(字符串模式)，实际应当保存为 `datetime` 类型



**整洁度**

1. `image_prediction_data` 数据集

	目前尚未发现需要调整的结构问题
	
2. `twitter_archive_data`	数据集

	* `source` 中保存了 `HTML` 标签类型数据
	* `text` 中保存了评分数据，`tweet` 文本信息以及链接数据
	* `expanded_urls` 存储了重复数据值
	* `field` 拆分为了多个，例如`doggo`,`floofer`, `pupper`, `puppo`

3. `tweet_data`	的数据集

	* `full_text` 中保存了评分数据，`tweet` 文本信息以及链接数据
	* `source` 中保存了 `HTML` 标签类型数据

In [ ]:
# create the data copy
image_data_copy = copy.deepcopy(image_prediction_data)
twitter_archive_data_copy = copy.deepcopy(twitter_archive_data)
tweet_data_copy = copy.deepcopy(tweet_data)

In [ ]:
print("Before wrangling data about the image data:\n")
print(image_data_copy.dtypes)

In [ ]:
# `tweet_id` 是 `int64` 的整数类型，实际应当保存为 `object` 类型
image_data_copy["tweet_id"] = image_data_copy["tweet_id"].astype("object")

In [ ]:
# `p1` 数据中首字母存在大写和小写混用的情况
# `p2` 数据中首字母存在大写和小写混用的情况
# `p3` 数据中首字母存在大写和小写混用的情况
image_data_copy[["p1", "p2", "p3"]] = image_data_copy[["p1", "p2", "p3"]].applymap(lambda x: x.title())

In [ ]:
# `p1_conf` 是比例数据，可以确认需要保留的小数点位数
# `p2_conf` 是比例数据，可以确认需要保留的小数点位数
# `p3_conf` 是比例数据，可以确认需要保留的小数点位数
image_data_copy[["p1_conf", "p2_conf", "p3_conf"]] = image_data_copy[["p1_conf", "p2_conf", "p3_conf"]].applymap(lambda x: round(x, 4))

In [ ]:
print("After wrangling data about the image data:\n")
print(image_data_copy.dtypes)

In [ ]:
print("Before wrangling data about the twitter archive data:\n")
print(twitter_archive_data_copy.dtypes)

In [ ]:
# `tweet_id` 是 `int64` 的整数类型，实际应当保存为 `object` 类型
twitter_archive_data_copy["tweet_id"] = twitter_archive_data_copy["tweet_id"].astype("object")

In [ ]:
# `in_reply_to_status_id` 是 `float64` 的整数类型，实际应当保存为 `object` 类型
# `in_reply_to_user_id` 是 `float64` 的整数类型，实际应当保存为 `object` 类型
# `retweeted_status_id` 是 `float64` 的数值类型数据，实际应当是 `object` 类型
# `retweeted_status_user_id` 是 `float64` 的数值类型数据，实际应当是 `object` 类型
twitter_archive_data_copy[["in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id"]] = \
    twitter_archive_data_copy[["in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id"]].applymap(
    lambda x: str(x))


In [ ]:
# `timestamp` 是 `object` 的对象数据(字符串模式)，实际应当保存为 `datetime` 类型
# `retweeted_status_timestamp` 是 `object` 的对象数据(字符串模式)，实际应当保存为 `datetime` 类型
twitter_archive_data_copy["timestamp"] = twitter_archive_data_copy["timestamp"].apply(pd.Timestamp)
twitter_archive_data_copy["retweeted_status_timestamp"] = twitter_archive_data_copy["retweeted_status_timestamp"].apply(pd.Timestamp)

In [ ]:
def html_content(tag):
    """
    get the data from the HTML tag. Return the content of the tag
    
    Args:
    (str) tag - the HTML tag
    
    Returns:
    (str) result - the content of the HTML tag
    """ 
    soup = BeautifulSoup(tag, "lxml")
    
    result = soup.get_text()
    
    return result

In [ ]:
# `source` 中保存了 `HTML` 标签类型数据
twitter_archive_data_copy["source"] = twitter_archive_data_copy["source"].apply(html_content)

source_dict = {"Twitter for iPhone":"iphone", 
               "Twitter Web Client": "web", 
               "Vine - Make a Scene": "vine",
               "TweetDeck": "deck"}

twitter_archive_data_copy["source"] = twitter_archive_data_copy["source"].map(source_dict)

In [ ]:
def text_split(text, tweet_text_option=False, rating_numerator_option=False, rating_denominator_option=False):
    """
    get the rate score from the text. Return the content of the tag
    
    Args:
    (str) text - the tweet text
    
    Returns:
    (tuple) result - the tuple contains the tweet text without the 
    rate score, rate scores
    """ 
    if not text:
        return text
    
    pattern = re.compile("\s{0,1}(\d{1,})/(\d{0,3}).{0,}|\s{.*\w\s{0,1}(https://.*\w){0,1}")

    match_pattern = re.search(pattern, text)
    
    # regex the text
    if match_pattern:
        rating_numerator = int(match_pattern.group(1))
        rating_denominator = int(match_pattern.group(2))

        tweet_text = re.sub(pattern, "", text).strip()
    else:
        rating_numerator = np.nan
        rating_denominator = np.nan
        tweet_text = text
    
    # return the result
    if tweet_text_option:
        return tweet_text

    if rating_denominator_option:
        return rating_denominator

    if rating_numerator_option:
        return rating_numerator

In [ ]:
# `text` 中保存了评分数据，`tweet` 文本信息以及链接数据
twitter_archive_data_copy["rating_denominator_1"] = twitter_archive_data_copy["text"].apply(lambda x: text_split(x, rating_denominator_option=True))
twitter_archive_data_copy["rating_numerator_1"] = twitter_archive_data_copy["text"].apply(lambda x: text_split(x, rating_numerator_option=True))
twitter_archive_data_copy["text"] = twitter_archive_data_copy["text"].apply(lambda x: text_split(x, tweet_text_option=True))

In [ ]:
# 验证提取到的 rating score 数据
print("The non_difference between the archive data and the extract data about the rating score:\n")
print("The non_difference about the rating denominator is %d" % \
      sum(twitter_archive_data_copy["rating_denominator"] == twitter_archive_data_copy["rating_denominator_1"]))
print("The non_difference about the rating numerator is %d" % \
      sum(twitter_archive_data_copy["rating_numerator"] == twitter_archive_data_copy["rating_numerator_1"]))

In [ ]:
# `expanded_urls` 存储了重复数据值
url = []
for head, value, tail in zip(np.repeat("https://twitter.com/dog_rates/status/", len(twitter_archive_data_copy)), 
                            twitter_archive_data_copy["tweet_id"], np.repeat("/photo/1", len(twitter_archive_data_copy))):
    url.append(head+str(value)+tail)

twitter_archive_data_copy["expanded_urls"] = pd.Series(url)

In [ ]:
#  `field` 拆分为了多个，例如`doggo`,`floofer`, `pupper`, `puppo`
dog_type = []
for doggo, floofer, pupper, puppo in zip(twitter_archive_data_copy["doggo"], 
                                         twitter_archive_data_copy["floofer"],
                                         twitter_archive_data_copy["pupper"],
                                         twitter_archive_data_copy["puppo"]):
    dog_type.append(doggo +  floofer + pupper + puppo)

In [ ]:
twitter_archive_data_copy["dog_type"] = pd.Series(dog_type)

twitter_archive_data_copy["dog_type"] = \
    twitter_archive_data_copy["dog_type"].apply(lambda x: x.replace("None", ""))

In [ ]:
twitter_archive_data_copy.dtypes

In [ ]:
# clean the duplicate data
twitter_archive_data_copy.drop(["doggo", "floofer", "pupper", "puppo", "rating_denominator_1", "rating_numerator_1"], axis=1, inplace=True)

In [ ]:
print("After wrangling data about the twitter archive data:\n")
print(twitter_archive_data_copy.dtypes)

In [ ]:
print("Before wrangling data about the tweet data:\n")
print(tweet_data_copy.dtypes)

In [ ]:
# `id` 是 `int64` 的整数类型，实际应当保存为 `object` 类型
tweet_data_copy["id"] = tweet_data_copy["id"].astype("object")

In [ ]:
#  `created_at` 是 `object` 的对象数据(字符串模式)，实际应当保存为 `datetime` 类型
tweet_data_copy["created_at"] = tweet_data_copy["created_at"].apply(pd.Timestamp)

In [ ]:
# `full_text` 中保存了评分数据，`tweet` 文本信息以及链接数据
tweet_data_copy["rating_denominator_1"] = tweet_data_copy["full_text"].apply(lambda x: int(text_split(x, rating_denominator_option=True)))
tweet_data_copy["rating_numerator_1"] = tweet_data_copy["full_text"].apply(lambda x: int(text_split(x, rating_numerator_option=True)))
tweet_data_copy["full_text"] = tweet_data_copy["full_text"].apply(lambda x: text_split(x, tweet_text_option=True))

In [ ]:
# `source` 中保存了 `HTML` 标签类型数据
tweet_data_copy["source"] = tweet_data_copy["source"].apply(html_content)

tweet_data_copy["source"] = tweet_data_copy["source"].map(source_dict)

In [ ]:
print("After wrangling data about the tweet data:\n")
print(tweet_data_copy.dtypes)

In [ ]:
tweet_data_copy["expanded_url"]

### 数据清洗后流程
以上经过数据清洗的过程，详情见 [wrangle_report](./wrangle_report.md)。完成以上的数据清洗，需要将相关数据进行融合：主要从几个方面去考虑，确立 **主键** ——因为需要依据主键值 `field` 来完成数据融合，另外还需要对各个 `field` 进行调整。在确立主键将数据融合到一起之后，需要再次对数据进行检查确认。至此才能保存数据结果。

In [ ]:
tweet_data_copy.dtypes

In [ ]:
twitter_archive_data_copy.dtypes

### 参考
1. [Tweet object — Twitter Developers](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html) 该文档对 `API` 数据的 `Field` 进行了解释，需要注意和目前得到的数据存在部分差异